In [1]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout
from keras.layers import Flatten
from keras.layers import Dense
# Implementing Callbacks to save various check points of our keras model; can be used to terminate model prematurely to also prevent overfitting
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
# for single predictions
from keras.preprocessing import image
import time

C:\Users\Appzone-MLG\Anaconda3\envs\chest_xray_env\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.callbacks import ModelCheckpoint, TensorBoard

In [3]:
# initializing the classfier
classifier = Sequential()

# Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (350, 350, 1), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2,2)))
classifier.add(Dropout(0.2))

classifier.add(Conv2D(64, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Dropout(0.2))

classifier.add(Conv2D(64, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Dropout(0.2))

classifier.add(Conv2D(64, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Dropout(0.2))

# Flattening
classifier.add(Flatten())

# Fully connected layer
classifier.add(Dense(activation = 'relu', units = 128))
classifier.add(Dropout(0.3))
classifier.add(Dense(activation = 'relu', units = 64))
classifier.add(Dropout(0.3))
classifier.add(Dense(activation = 'relu', units = 64))
classifier.add(Dropout(0.3))
classifier.add(Dense(activation = 'relu', units = 64))
classifier.add(Dropout(0.3))
classifier.add(Dense(activation = 'relu', units = 64))
classifier.add(Dropout(0.3))
classifier.add(Dense(activation = 'sigmoid', units = 14))

classifier.compile(optimizer = 'adam',
                   loss ='binary_crossentropy',
                   metrics = ['accuracy']
                  )






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [4]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 348, 348, 32)      320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 174, 174, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 174, 174, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 172, 172, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 86, 86, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 86, 86, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 84, 84, 64)        36928     
__________

In [5]:
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.preprocessing import image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
%matplotlib inline

In [6]:
# to display all the columns of the dataframe in the notebook
pd.pandas.set_option('display.max_columns', None)

In [7]:
# Reading the csv file 
# Here we will be using a subset of the original dataset

train = pd.read_csv('CheXpert-v1.0-small/train_data.csv', nrows=1000)

In [8]:
train.head()

,path,no_finding,enlarged_cardiomediastinum,cardiomegaly,lung_opacity,lung_lesion,edema,consolidation,pneumonia,atelectasis,pneumothorax,pleural_effusion,pleural_other,fracture,support_devices,no_of_labels
0,CheXpert-v1.0-small/train/patient00001/study1/...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,CheXpert-v1.0-small/train/patient00002/study2/...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0
2,CheXpert-v1.0-small/train/patient00002/study1/...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0
3,CheXpert-v1.0-small/train/patient00002/study1/...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0
4,CheXpert-v1.0-small/train/patient00003/study1/...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [9]:
# loading and preprocessing 
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img(''+train['path'][i]+'', target_size=(350, 350, 1), color_mode = "grayscale")
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 430.70it/s]


In [10]:
# X.shape

In [11]:
# X[0]

In [12]:
# plt.imshow(X[0], cmap='gray')
# plt.imshow(X[2], cmap='gray')

In [13]:
# Extracting the Onehot encoded label
y = np.array(train.drop(['path', 'no_of_labels'],axis=1))
y.shape

(1000, 14)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.1)

In [15]:
classifier.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=32)

Train on 900 samples, validate on 100 samples
Epoch 1/50
900/900 [==============================] - 11s 12ms/step - loss: 0.5054 - acc: 0.7754 - val_loss: 0.5448 - val_acc: 0.8200
Epoch 2/50
900/900 [==============================] - 6s 6ms/step - loss: 0.4168 - acc: 0.8240 - val_loss: 0.5293 - val_acc: 0.8200
Epoch 3/50
900/900 [==============================] - 6s 6ms/step - loss: 0.4063 - acc: 0.8270 - val_loss: 0.4980 - val_acc: 0.8157
Epoch 4/50
900/900 [==============================] - 6s 6ms/step - loss: 0.4014 - acc: 0.8325 - val_loss: 0.4658 - val_acc: 0.8193
Epoch 5/50
900/900 [==============================] - 6s 6ms/step - loss: 0.3964 - acc: 0.8275 - val_loss: 0.4204 - val_acc: 0.8093
Epoch 6/50
900/900 [==============================] - 6s 6ms/step - loss: 0.3900 - acc: 0.8314 - val_loss: 0.4140 - val_acc: 0.8157
Epoch 7/50
900/900 [==============================] - 6s 6ms/step - loss: 0.3867 - acc: 0.8310 - val_loss: 0.3944 - val_acc: 0.8157
Epoch 8/50
900/900 [========

In [16]:
# Using image generators
from keras.preprocessing.image import ImageDataGenerator

In [31]:
train_df = pd.read_csv("CheXpert-v1.0-small/train_data.csv", nrows=6400)
valid_df = pd.read_csv("CheXpert-v1.0-small/valid.csv")

tensor_log_name = f"chest-xray-cnn-{int(time.time())}"

train_datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.2,
        horizontal_flip=False)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=None,
        x_col="path",
        y_col=['no_finding',
               'enlarged_cardiomediastinum',
               'cardiomegaly',
               'lung_opacity',
               'lung_lesion',
               'edema',
               'consolidation',
               'pneumonia',
               'atelectasis',
               'pneumothorax',
               'pleural_effusion',
               'pleural_other',
               'fracture',
               'support_devices'
               ],
        color_mode="grayscale",
        target_size=(350, 350),
        batch_size=32,
#         class_mode="multi_output",
        class_mode="raw",
)

validation_generator = test_datagen.flow_from_dataframe(
        dataframe=valid_df,
        directory=None,
        x_col="Path",
        y_col=['No Finding',
               'Enlarged Cardiomediastinum',
               'Cardiomegaly',
               'Lung Opacity',
               'Lung Lesion',
               'Edema',
               'Consolidation',
               'Pneumonia',
               'Atelectasis',
               'Pneumothorax',
               'Pleural Effusion',
               'Pleural Other',
               'Fracture',
               'Support Devices'
              ],
        color_mode="grayscale",
        target_size=(350, 350),
        batch_size=128,
        class_mode="raw")

# filepath contains location for storing keras models with the lowwest loss function after every 10 epoch
filepath="models/weights-improvement-{epoch:02d}-{val_loss:.2f}-{val_acc:.2f}.hdf5"

checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='min',
#                              save_freq=5
                             period=2
                            )

tensorboard = TensorBoard(log_dir = f'logs/{tensor_log_name}')

classifier.fit_generator(
        train_generator,
        steps_per_epoch=200,
        epochs=15,
        validation_data=validation_generator,
        validation_steps=2,
        callbacks=[checkpoint, tensorboard]
)

Found 6400 validated image filenames.
Found 234 validated image filenames.
Epoch 1/15
200/200 [==============================] - 77s 387ms/step - loss: 0.3311 - acc: 0.8591 - val_loss: 0.4457 - val_acc: 0.8068
Epoch 2/15
200/200 [==============================] - 75s 374ms/step - loss: 0.3295 - acc: 0.8607 - val_loss: 0.4476 - val_acc: 0.8111

Epoch 00002: val_loss improved from inf to 0.44764, saving model to models/weights-improvement-02-0.45-0.81.hdf5
Epoch 3/15
200/200 [==============================] - 74s 369ms/step - loss: 0.3278 - acc: 0.8609 - val_loss: 0.4454 - val_acc: 0.8080
Epoch 4/15
200/200 [==============================] - 77s 384ms/step - loss: 0.3265 - acc: 0.8621 - val_loss: 0.4465 - val_acc: 0.8141

Epoch 00004: val_loss improved from 0.44764 to 0.44653, saving model to models/weights-improvement-04-0.45-0.81.hdf5
Epoch 5/15
200/200 [==============================] - 75s 375ms/step - loss: 0.3232 - acc: 0.8634 - val_loss: 0.4512 - val_acc: 0.8114
Epoch 6/15
200/200

In [19]:
# Single preds
test_image = image.load_img('CheXpert-v1.0-small/valid/patient64740/study1/view1_frontal.jpg',
                            target_size = (350, 350),
                            color_mode= 'grayscale'
                           )

In [20]:
print(test_image)

<PIL.Image.Image image mode=L size=350x350 at 0x1F5FC521EF0>


In [21]:
test_image_arr = image.img_to_array(test_image)

In [22]:
# test_image_arr.shape

In [23]:
test_image_arr = np.expand_dims(test_image_arr, axis = 0)

In [24]:
classifier.predict(test_image_arr)

array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 2.3841858e-07,
        0.0000000e+00, 7.5799823e-03, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 1.0000000e+00]], dtype=float32)

In [25]:
# Save model on local storage

# Save the model
classifier.save('models/first_test_model.h5')

# Recreate the exact same model purely from the file
new_model = keras.models.load_model('models/first_test_model.h5')

In [ ]:
# TensorBoard(log_dir='./logs',
#              histogram_freq=0,
#              batch_size=32,
#              write_graph=True,
#              write_grads=False,
#              write_images=False,
#              embeddings_freq=0,
#              embeddings_layer_names=None,
#              embeddings_metadata=None,
#              embeddings_data=None,
#              update_freq='epoch'
#            )

In [ ]:
# run in cmd
# tensorboard --logdir=/logs

# Enter link below in browser to visit TensorBoard
http://localhost:6006/